In [1]:
# Energy
idNo = '271090'

In [2]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd
import json
import csv

In [3]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [4]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
# searchTerm1 = ['Building Information Modelling']
# searchTerm2 = ['Green Buildings']

# searchTerm1 = ['BIM']
# searchTerm2 = ['Sustainability']

tc = 0

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "%20"

    fileIndex = text2.rfind("%20")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "%20"

    fileIndex = text1.rfind("%20")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

['Green%20Building', 'Green%20Buildings', 'Sustainable%20building', 'Sustainable%20buildings', 'Environmentally%20friendly%20design', 'Sustainability', 'Sustainable%20development', 'Energy', 'Water', 'Waste', 'Thermal%20comfort', 'Electricity', 'Solar', 'Wind%20power', 'Green%20infrastructure', 'Life%20cycle%20assessment']
['BIM', 'Building%20Information%20Modelling', 'Building%20information%20model', '6D%20BIM', 'Digital%20construction', 'Smart%20construction', 'Digital%20engineering']


In [6]:
def getData(link):
    browser = webdriver.Safari()
    browser.get(link)
    respData = browser.page_source
    browser.quit()
    parsed_html = BeautifulSoup(respData)
    
    title = parsed_html.find('span',{'class':'title-text'}).text
    print("Title: ",title)
    df.loc[row,'url'] = link
    df.loc[row,'Title'] = title
    
    absResults = parsed_html.find('div',{'class':'abstract author'})
    
    if absResults is not None:
        try:            
            df.loc[row,'section0'] = absResults.find('h2').text
            df.loc[row,'sectionData0'] = absResults.find('div').text
        except:
            df.loc[row,'section0'] = "None"
            df.loc[row,'sectionData0'] = "None"
    else:
        df.loc[row,'section0'] = "None"
        df.loc[row,'sectionData0'] = "None"

        
    sections = parsed_html.find('div',{'class':'Body u-font-serif'})
    if sections is not None:        
        sections = sections.find_next('div')
        count = 1 
        for section in sections:
            if section is not None: 
                sectionNo = 'section' + str(count)
                sectionData = 'sectionData' + str(count)
                try:
                    df.loc[row,sectionNo] = section.find('h2',{'class':'u-h3 u-margin-l-top u-margin-xs-bottom'}).text
                except:
                    df.loc[row,sectionNo] = "Some issue"
                    
                data = ''
                for para in section.findAll('p'):
                    data = data + para.text
                df.loc[row,sectionData] = data
            else:
                df.loc[row,sectionNo] = "None"
            count = count + 1        

In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [8]:
baseUrl = 'https://www.sciencedirect.com/search/api?tak=%22'

querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        sq = 'https://www.sciencedirect.com/search?qs=%22' + item1 + '%22%20%2B%20%22' + item2 + '%22'
        print("base query: ",sq)
        query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
        
        # Call webdriver         
        browser = webdriver.Safari()
        browser.get(query)
        respData = browser.page_source
        browser.quit()

        # Clear html         
        respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
        respData = respData.replace('</pre></body></html>',"")
        
        py_dic = json.loads(respData)
        
        # Finding the number of pages in the selected journals.         
        for key in py_dic:
            if key == 'resultsFound':
                pageResults = py_dic[key]
        print("pageResults: ",pageResults)                
                                
        if pageResults > 25:
            page = pageResults // 25
            lastPage = pageResults % 25
            if lastPage > 0:
                page = page + 1
        else:
            page = 1
        print("page: ",page)

        #         for p in range(2):        
        for p in range(page):
            print("In page: ",p)
            if p == 0:
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
            else:
                offset = '&offset='+str(p*25)
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&show=25&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
                
            print("search query: ",query)  
                
            # Call webdriver         
            browser = webdriver.Safari()
            browser.get(query)
            respData = browser.page_source
            browser.quit()
            # Load Json Data 

            respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
            respData = respData.replace('</pre></body></html>',"")

            py_dic = json.loads(respData)                
            # Quit the browser 
            links = []
            for key in py_dic:
                if key == 'searchResults':
                    for searchR in py_dic[key]:
                        for search in searchR:
                            if search == 'pdf':
                                for linkR in searchR[search]:
                                    if linkR == 'getAccessLink':
                                        baseLink = 'https://www.sciencedirect.com/'
                                        link = baseLink +searchR[search][linkR]
                                        print('Reading result: ',row)
                                        df.loc[row,'searchTerm1'] = item1
                                        df.loc[row,'searchTerm2'] = item2
                                        getData(link)                                    
                                        row = row + 1        

base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20Building%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Green%20Building%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  0
Title:  An investigation of the impact of building orientation on energy consumption in a domestic building using emerging BIM (Building Information Modelling)
base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20Buildings%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Green%20Buildings%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslI

base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20infrastructure%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Green%20infrastructure%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Life%20cycle%20assessment%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22BIM%22%20%2B%20%22Life%20cycle%20assessment%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query: 

pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20Information%20Modelling%22%20%2B%20%22Thermal%20comfort%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Building%20Information%20Modelling%22%20%2B%20%22Electricity%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Building%20Information%20Modelling%22%20%2B%20%22Electricity%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  23
Title:  An investigation of the 

Reading result:  28
Title:  An investigation of the impact of building orientation on energy consumption in a domestic building using emerging BIM (Building Information Modelling)
Reading result:  29
Title:  Automated Building Energy Modeling and Assessment Tool (ABEMAT)
Reading result:  30
Title:  Building occupancy diversity and HVAC (heating, ventilation, and air conditioning) system energy efficiency
Reading result:  31
Title:  Assessment of the building components in the energy efficient design of tropical residential buildings: An application of BIM and statistical Taguchi method
Reading result:  32
Title:  Information needs for progressive BIM methodology supporting the holistic energy renovation of office buildings
Reading result:  33
Title:  Multi-criterion optimization of building envelope in the function of indoor illumination quality towards overall energy performance improvement
base query:  https://www.sciencedirect.com/search?qs=%22Building%20information%20model%22%20%2B

base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Sustainable%20development%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%226D%20BIM%22%20%2B%20%22Sustainable%20development%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Energy%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%226D%20BIM%22%20%2B%20%22Energy%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www

base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Energy%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20construction%22%20%2B%20%22Energy%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Water%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20construction%22%20%2B%20%22Water%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base que

base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Energy%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Smart%20construction%22%20%2B%20%22Energy%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Water%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Smart%20construction%22%20%2B%20%22Water%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  htt

base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Energy%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20engineering%22%20%2B%20%22Energy%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Water%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?tak=%22Digital%20engineering%22%20%2B%20%22Water%22&show=25&publicationTitles=271090&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCBn%252BbzsV301l1vJIS08F%252FfZyGd1BvBslIQLVrbHXEzqKNF6BecRyI0u5A30hykv1MD5FCWUf4DF7Nbn6utpsZ0muydTQtxvTP56QYrBeyn6WvKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query: 

In [9]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,An investigation of the impact of building ori...,Abstract,BIM (building information modelling) has devel...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BIM,Green%20Buildings,https://www.sciencedirect.com//science/article...,An investigation of the impact of building ori...,Abstract,BIM (building information modelling) has devel...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BIM,Sustainability,https://www.sciencedirect.com//science/article...,Information needs for progressive BIM methodol...,Abstract,The paper presents digital methodology that ca...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Sustainability,https://www.sciencedirect.com//science/article...,An investigation of the impact of building ori...,Abstract,BIM (building information modelling) has devel...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BIM,Energy,https://www.sciencedirect.com//science/article...,Information needs for progressive BIM methodol...,Abstract,The paper presents digital methodology that ca...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BIM,Energy,https://www.sciencedirect.com//science/article...,Assessment of the building components in the e...,Abstract,"With the growing energy and economic crises, t...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BIM,Energy,https://www.sciencedirect.com//science/article...,An investigation of the impact of building ori...,Abstract,BIM (building information modelling) has devel...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BIM,Energy,https://www.sciencedirect.com//science/article...,Automated Building Energy Modeling and Assessm...,Abstract,Buildings contribute to about 40% of total ann...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BIM,Energy,https://www.sciencedirect.com//science/article...,Building occupancy diversity and HVAC (heating...,Abstract,Approximately forty percent of total building ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BIM,Energy,https://www.sciencedirect.com//science/article...,Multi-criterion optimization of building envel...,Abstract,This paper elaborates the formulation and appl...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df1 = df
df['journalName'] = str(idNo)
fileSave = 'kgRawData/SD/sd-kg'+idNo+'.csv'
df.to_csv(fileSave)

In [1]:
idNo = '271090'

import pandas as pd
# filePath = 
data = pd.read_csv('kgRawData/SD/sd-kg' + idNo + '.csv')
data['presence'] = False

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

validEntries = []

for i in range(data.shape[0]):    
    for st1 in searchTerm1:
        for st2 in searchTerm2:
            if (st1 in data['sectionData0'][i]) and (st2 in data['sectionData0'][i]):
                data['presence'][i] = True
                validEntries.append(i)

uniqueValidEntries = list(set(validEntries))

uniqueDf = pd.DataFrame()
uniqueDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

uniqueDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
uniqueDf = uniqueDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    uniqueDf[section] = ''
    uniqueDf[sectionData] = ''
    
for entry in range(len(uniqueValidEntries)):
    uniqueDf.loc[entry] = data.loc[uniqueValidEntries[entry]]    
    

uniqueDf.to_csv('kgRawData/SD/cleanSd-kg'+ idNo + '.csv')

print(len(uniqueDf))

24


<ipython-input-1-71a9685b39bb>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['presence'][i] = True


In [3]:
uniqueDf['url'][0]

'https://www.sciencedirect.com//science/article/pii/S0360544216000037'